In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType

# Start Spark session
spark = SparkSession.builder.appName("InsuranceData").getOrCreate()

# Define schema
schema = StructType([
    StructField("pid", IntegerType(), True),
    StructField("tiv_2015", DoubleType(), True),
    StructField("tiv_2016", DoubleType(), True),
    StructField("lat", DoubleType(), True),
    StructField("lon", DoubleType(), True)
])

# Create data
data = [
    (1, 10.0, 5.0, 10.0, 10.0),
    (2, 20.0, 20.0, 20.0, 20.0),
    (3, 10.0, 30.0, 20.0, 20.0),
    (4, 10.0, 40.0, 40.0, 40.0)
]

# Create DataFrame
df = spark.createDataFrame(data, schema)

# Show the DataFrame
df.show()


In [0]:

select ROUND(sum(tiv_2016),2) as tiv_2016
from (
select *,
count(*) over(partition by tiv_2015) as cnt1,
count(*) over(partition by lat,lon) as cnt2
from insurance
) a
where cnt1 >=  2 and cnt2 = 1

In [0]:
w1 = Window.partitionBy('tiv_2015')
w2 = Window.partitionBy('lat','lon')

In [0]:
df1 = df.withColumn('cnt1', count('*').over(w1)).withColumn('cnt2', count('*').over(w2)).orderBy('pid')
result_df = df1.filter('cnt1 >= 2' and 'cnt2==1').agg(round(sum(col('tiv_2016')),2).alias('tiv_2016'))
result_df.show()